In [1]:

from pathlib import Path
from pprint import pprint

from ReChisel.testcase import Testcase
from ReChisel.chisel_code import ChiselCode
from ReChisel.verifier import VerifyResult, verify
from ReChisel.generator import Generator

generator = Generator(
    init_gen_model='gpt-4o',
    init_gen_system_prompt=Path('prompts/chisel_generation.txt').read_text(encoding='utf-8'),

    # This code snippet does not do correction generation.
    correction_model='na',
    syntax_correction_system_prompt='Unused',
    functionality_correction_system_prompt='Unused',

    verbose=True,
)


/home/niujuxin/anaconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_arn" in BedrockRerank has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
def code_gen_and_verify(bmcase: Testcase, top_module_name: str, bm_type: str):

    print('==' * 20, "Specification", '==' * 20)
    print(bmcase.specification)
    print()

    print('==' * 20, "Chisel Code Generation", '==' * 20)
    generator.testcase_prepare(bmcase, top_module_name)
    response = generator.initial_chisel_generation()
    chisel_code: ChiselCode = generator.code_extract(response)
    print('-' * 20, "Generated Chisel Code", '-' * 20)
    print(chisel_code.raw)
    print()

    print('==' * 20, "Verification", '==' * 20)
    verify_result: VerifyResult = verify(
        chisel_code, bmcase,
        output_dir=Path('output/verifier/'),
        bm_type=bm_type,
        verbose=True
    )
    print('-' * 20, "Verification Result", '-' * 20)
    _d = verify_result.__dict__()
    _d.pop('compiled_verilog_code', None)  # Remove verilog code for better readability
    pprint(_d)
    

In [3]:

bmcase = Testcase(
    'Prob030_popcount255', 
    'benchmarks/VerilogEval_Prob030/Prob030_popcount255_spec.txt',
    'benchmarks/VerilogEval_Prob030/Prob030_popcount255_ref.sv',
    'benchmarks/VerilogEval_Prob030/Prob030_popcount255_tb.sv'
)

code_gen_and_verify(bmcase, 'TopModule', bm_type='verilog-eval')


======================================== Specification ========================================
I would like you to implement a module named TopModule with the following
interface. All input and output ports are one bit unless otherwise
specified.

 - input  in  (255 bits)
 - output out (  8 bits)

A "population count" circuit counts the number of '1's in an input
vector. The module should implement a population count circuit for a
255-bit input vector.

======================================== Chisel Code Generation ========================================
[GENERATOR] Preparing testcase for code generation.
[GENERATOR] Testcase:Testcase(prob_id=Prob030_popcount255, specification_dir=benchmarks/VerilogEval_Prob030/Prob030_popcount255_spec.txt, reference_dir=benchmarks/VerilogEval_Prob030/Prob030_popcount255_ref.sv, testbench_dir=benchmarks/VerilogEval_Prob030/Prob030_popcount255_tb.sv)
[GENERATOR] Top module name: TopModule
[GENERATOR] Preparing messages for initial Chisel code generat

In [4]:

bmcase = Testcase(
    'Prob001_zero', 
    'benchmarks/VerilogEval_Prob001/Prob001_zero_spec.txt',
    'benchmarks/VerilogEval_Prob001/Prob001_zero_ref.sv',
    'benchmarks/VerilogEval_Prob001/Prob001_zero_tb.sv'
)

code_gen_and_verify(bmcase, 'TopModule', bm_type='verilog-eval')


======================================== Specification ========================================

I would like you to implement a module named TopModule with the following
interface. All input and output ports are one bit unless otherwise
specified.

 - output zero

The module should always outputs a LOW.



======================================== Chisel Code Generation ========================================
[GENERATOR] Preparing testcase for code generation.
[GENERATOR] Testcase:Testcase(prob_id=Prob001_zero, specification_dir=benchmarks/VerilogEval_Prob001/Prob001_zero_spec.txt, reference_dir=benchmarks/VerilogEval_Prob001/Prob001_zero_ref.sv, testbench_dir=benchmarks/VerilogEval_Prob001/Prob001_zero_tb.sv)
[GENERATOR] Top module name: TopModule
[GENERATOR] Preparing messages for initial Chisel code generation.
[GENERATOR] Calling LLM for initial Chisel code generation with model gpt-4o.
[GENERATOR] Initial Chisel code generation response received.
[GENERATOR] Chisel code extracted

In [6]:

bmcase = Testcase(
    'AutoChip_Vector5', 
    'benchmarks/AutoChip_Vector5/Vector5_spec.txt',
    None,
    'benchmarks/AutoChip_Vector5/Vector5_0_tb.v'
)

code_gen_and_verify(bmcase, 'TopModule', bm_type='autochip')


======================================== Specification ========================================
I would like you to implement a module named `TopModule` with the following interface. 
All input and output ports are one bit wide unless otherwise specified.

Module Name: `TopModule`
Ports:
  input a,
  input b,
  input c,
  input d,
  input e,
  output [24:0] out

Given five 1-bit signals (a, b, c, d, and e), compute all 25 pairwise one-bit comparisons in the 25-bit output vector. 
The output should be 1 if the two bits being compared are equal.

Hint: 
out[24] = ~a ^ a;   // a == a, so out[24] is always 1.
out[23] = ~a ^ b;
out[22] = ~a ^ c;
...

======================================== Chisel Code Generation ========================================
[GENERATOR] Preparing testcase for code generation.
[GENERATOR] Testcase:Testcase(prob_id=AutoChip_Vector5, specification_dir=benchmarks/AutoChip_Vector5/Vector5_spec.txt, reference_dir=None, testbench_dir=benchmarks/AutoChip_Vector5/Vector5